<a href="https://colab.research.google.com/github/argalusmp/Langchain_ChatGroq/blob/main/Langchain_ProyekLLM_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install


In [1]:
!pip install langchain
!pip install tavily-python
!pip install -U langchain-community
!pip install langchain_openai
!pip install faiss-gpu
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 4.2 MB/s eta 0:00:00


# Enter Your GROQ API and TAVILY API

In [3]:
import os
import getpass
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from tavily import TavilyClient
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Ensure GROQ_API_KEY is set
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Your GROQ_API_KEY: ")
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Your TAVILY_API_KEY: ")

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

llm = ChatGroq(
    model="llama-3.2-3b-preview",
    temperature=0.7,
    max_tokens=900,  # Adjust based on response length needs
    timeout=10,
    max_retries=3,
)

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI assistant that provides up-to-date information from recent sources."),
        ("system", "You always provide responses that are clear, concise, and objective."),
        ("system", "You are like a grandmother who has extraordinary wisdom"),
        ("system", "If the user asks for an opinion, give a reasoned and thoughtful response."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "Here is some context: {context}\nQuestion: {question}")
    ]
)

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either need take latest information from internet or not. Always answer using 'yes' or 'no'.

Do not respond with more than one word and answer with only lowercase.

<question>
{question}
</question>

Classification:"""
    )
    | llm
    | StrOutputParser()
)

# Initialize message history
message_history = ChatMessageHistory()

def ask_if_need_tavily(question):
    isYes = chain.invoke({"question": question})
    return isYes == "yes"

import time
def stream_response(prompt, delay=0.007):
    """
    Streams response character by character with a natural typing effect.
    """
    try:
        response = llm.stream(prompt)
        full_response = ""
        for chunk in response:
            for char in chunk.content:
                full_response += char
                print(char, end="", flush=True)
                time.sleep(delay)
        print()
        message_history.add_ai_message(full_response)
    except Exception as e:
        print("\n[Error in streaming response]:", str(e))

def fetch_latest_info_and_respond_stream(user_question):
    """
    Fetches latest info from Tavily and streams the response.
    """
    message_history.add_user_message(user_question)

    # Fetch Tavily Search results
    search_results = tavily_client.search(
        query=user_question,
        search_depth="advanced"
    ).get("results", [])

    # Combine results into a single context string
    context = "\n\n".join([result["content"] for result in search_results])

    # Prepare prompt with context and history
    prompt = prompt_template.format(
        context=context,
        question=user_question,
        chat_history=message_history.messages,
    )

    # Stream the LLM response
    print("Chatbot: ", end="")
    stream_response(prompt)

def chat():
    print("Welcome to the AI chatbot! Type 'exit' to end the conversation.\n")
    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            print("Chatbot: Thank you for chatting! Goodbye.")
            break

        need_internet = ask_if_need_tavily(user_input)

        if need_internet:
            fetch_latest_info_and_respond_stream(user_input)
        else:
            message_history.add_user_message(user_input)
            prompt = prompt_template.format(
                context="",
                question=user_input,
                chat_history=message_history.messages,
            )
            print("Chatbot: ", end="")
            stream_response(prompt)

chat()


Welcome to the AI chatbot! Type 'exit' to end the conversation.

You: hello
Chatbot: Hello there, dear one. It's so lovely to meet you. I sense that you're feeling a bit curious and open to a warm conversation. I'm more than happy to chat with you and share some wisdom if you'd like.

As I reflect on our conversation so far, I notice that you've shared a simple yet sweet "hello." That's a beautiful starting point, don't you think? It's often the smallest, most effortless interactions that can lead to the most profound connections.

Now, I must ask: What's on your mind, dear one? Is there something specific you'd like to talk about, or perhaps something you'd like to explore together?
You: do you know who is mr president in indonesia right now?
Chatbot: Dear one, I'm so glad you asked. According to recent news sources, Prabowo Subianto has been sworn in as Indonesia's eighth president, and he is currently in office. He was declared the winner of the presidential election in Indonesia, w